Since linear models don't capture interactions between features, here, we test for this. This can reveal synergies or antagonisms that have outsized effects on metastatic potential. 

Methodologically, we fit an OLS on regressing the residuals of the model error on the interaction between two features. 

In [1]:
# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--feature_type", type=str, required=True, help="how to filter features")
# args = parser.parse_args()
# feature_type = args.feature_type

# # python 05_perturbation_interactions_residuals_sets.py --feature_type all 
# # feature_type = 'all' # 'top400', 'enriched'

In [5]:
import os
import itertools
import random
from multiprocessing import Pool
from math import comb

from tqdm import tqdm 

import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from sklearn.base import clone


import seaborn as sns
import matplotlib.pyplot as plt 

import sys
sys.path.insert(1, '../../')
from utils import read_pickled_object

In [2]:
data_path = '/home/hmbaghda/orcd/pool/metastatic_potential/'
random_state = 42 + 3

n_cores = 60
os.environ["OMP_NUM_THREADS"] = str(n_cores)
os.environ["MKL_NUM_THREADS"] = str(n_cores)
os.environ["OPENBLAS_NUM_THREADS"] = str(n_cores)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(n_cores)
os.environ["NUMEXPR_NUM_THREADS"] = str(n_cores)

# Preprocess

Load data:

In [3]:
X = pd.read_csv(os.path.join(data_path, 'processed',  'expr_joint.csv'), index_col = 0)
expr_joint = X.copy()

mp_joint=pd.read_csv(os.path.join(data_path, 'processed', 'metastatic_potential_joint.csv'), index_col = 0)['mean']
y = mp_joint.values.ravel()

expr_protein = pd.read_csv(os.path.join(data_path, 'processed',  'expr_protein.csv'), index_col = 0)
expr_rna = pd.read_csv(os.path.join(data_path, 'processed',  'expr.csv'), index_col = 0)

protein_cols = expr_protein.columns
rna_cols = expr_rna.columns

X_protein = X[protein_cols].values
X_rna = X[rna_cols].values


As in [Notebook 04](./04_feature_analysis.ipynb), we fit the consensus linear SVR on the entire dataset:

In [6]:
best_pipeline = read_pickled_object(os.path.join(data_path, 'processed', 
                                                 'best_model_svr_linear_joint.pickle'))

X = (X_protein, X_rna)
best_pipeline = clone(best_pipeline)
best_pipeline.fit(X, y)

# load from notebook 04
# best_pipeline = read_pickled_object(os.path.join(data_path, 'interim', 'best_linearSVR_joint_fitted_allsamples.pickle'))


,steps,"[('feature_processing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformer_list,"[('protein_pipeline', ...), ('rna_pipeline', ...)]"
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,modality,'protein'
,method,'top_residuals'


In [7]:
model_coefs = pd.read_csv(os.path.join(data_path, 'interim', 'joint_features.csv'), 
                          index_col = 0)
if not np.allclose(model_coefs['SVM coefficient'].values, 
                   best_pipeline.named_steps['model'].coef_.flatten()):
    raise ValueError('Inconsitency between Notebook 04 and 05')
model_coefs.sort_values(by='SVM coefficient', key=lambda x: x.abs(), ascending=False, inplace=True)
model_coefs.set_index('feature_name', inplace = True)


Get the prediction:

In [8]:
# get prediction
y_pred = best_pipeline.predict(X)
residuals = y - y_pred

/orcd/pool/005/hmbaghda/miniforge3/envs/metastatic_potential/lib/python3.11/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/orcd/pool/005/hmbaghda/miniforge3/envs/metastatic_potential/lib/python3.11/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [9]:
X_map = {'Transcriptomics': X_rna, 'Proteomics': X_protein}
X_map = {k: X_ - np.mean(X_, axis=0) for k, X_ in X_map.items()} # center the data

# # center and scale the data
# from sklearn.preprocessing import StandardScaler
# for k, X_ in X_map.items():
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X_)
#     X_map[k] = X_scaled
    
# for k, X_ in X_map.items():
#     X_centered = X_ - np.mean(X_, axis=0) # mean center
#     X_map[k] = X_centered
#     X_scaled = X_centered / np.std(X_, axis=0) # scale
#     X_map[k] = X_scaled

# Top Genes

Let's see whether there are interactions between the top 25, 50, or 100 features ranked by the model coefficient:

In [10]:
def get_interaction_value(feature_1: str, feature_2: str, residuals, X_map, model_coefs):
    X_1 = X_map[model_coefs.loc[feature_1,'Modality']][:, model_coefs.loc[feature_1, 'feature_index']]
    X_2 = X_map[model_coefs.loc[feature_2,'Modality']][:, model_coefs.loc[feature_2, 'feature_index']]

    ols_df = pd.DataFrame({"residual": residuals, 
                           "X_tilda": X_1 * X_2})

    ols_interaction = smf.ols("residual ~ X_tilda", data=ols_df).fit()


    coef = float(ols_interaction.params.X_tilda)
    pval = float(ols_interaction.pvalues.X_tilda)
    
    se = ols_interaction.bse.X_tilda
    
    t_statistic = float(abs(coef))/se
    
    return coef, pval, t_statistic

In [11]:
alpha = 0.2
top_n_res = {}
for top_n in [25, 50, 100]:
    features = model_coefs.index.tolist()[:top_n]
    feature_combs = list(itertools.combinations(features, 2))
    
    res_all = []
    for feature_comb in tqdm(feature_combs):
        res = get_interaction_value(feature_comb[0], feature_comb[1], residuals, X_map, model_coefs)[:-1]
        res_all.append(res)

    res_all = pd.DataFrame(res_all, columns = ['coef', 'pval'])
    _, bh_fdr, _, _ = multipletests(res_all.pval, method='fdr_bh')
    res_all['bh_fdr'] = bh_fdr

    res_all = pd.concat([
        pd.DataFrame(feature_combs, columns = ['feature_1', 'feature_2']),
        res_all
    ],
        axis = 1)
    
    n_sig_hits = res_all[res_all.bh_fdr <= alpha].shape[0]
    
    print('Considering the top {} genes, there are {} significant interactions out of a possible {}'.format(top_n, 
                                                                                                            n_sig_hits, 
                                                                                                            comb(top_n, 2)))

    top_n_res[top_n] = res_all

100%|████████████████████████████████████████| 300/300 [00:00<00:00, 323.22it/s]


Considering the top 25 genes, there are 0 significant interactions out of a possible 300


100%|██████████████████████████████████████| 1225/1225 [00:02<00:00, 464.85it/s]


Considering the top 50 genes, there are 0 significant interactions out of a possible 1225


100%|██████████████████████████████████████| 4950/4950 [00:10<00:00, 464.63it/s]

Considering the top 100 genes, there are 0 significant interactions out of a possible 4950


We see that amongst the top ranked genes, there are no significant interactions. However, it is possible for interactions to occur non-intuitively across features that don't have high ranks. So, in the next notebook, let's use a genetic algorithm identify highly interacting genes. 